In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import os
from ast import literal_eval
from PIL import Image, ImageDraw

In [ ]:
df = pd.read_csv('/kaggle/input/tensorflow-great-barrier-reef/train.csv')
df.head()

In [ ]:
df['old_image_path'] = os.path.join('/kaggle/input/tensorflow-great-barrier-reef/train_images/')+'video_'+(df['video_id'].astype(str))+'/'+(df['video_frame'].astype(str))+'.jpg'
df.head()

In [ ]:
df['annotations'] = df['annotations'].apply(literal_eval)
df.head()

In [ ]:
df['Num_of_Bounding_boxes'] = df['annotations'].apply(lambda x: len(x))
df.head()

In [ ]:
def area_calc(bbox_lst):
    area = 0
    for bbox in bbox_lst:
        area = area + bbox['height']*bbox['width']
    return area

In [ ]:
df['area_bbox'] = df['annotations'].apply(lambda x: area_calc(x))
df.sort_values('area_bbox',ascending=False).head(2)

In [ ]:
df_train = df[df['Num_of_Bounding_boxes']>0].reset_index().drop(columns=['index'])
print(df_train.shape)

In [ ]:
import random
Is_Training = random.choices(['train','val'],weights=[70,30],k = df_train.shape[0])
df_train['Is_Train'] = pd.Series(Is_Training)
df_train['new_image_path'] = os.path.join('/kaggle/working/')+df_train['Is_Train']+'/images/'+(df_train['video_id'].astype(str))+'-'+(df_train['video_frame'].astype(str))+'.jpg'
df_train['new_label_path'] = os.path.join('/kaggle/working/')+df_train['Is_Train']+'/labels/'+(df_train['video_id'].astype(str))+'-'+(df_train['video_frame'].astype(str))+'.txt'
df_train.head(2)

In [ ]:
os.chdir('/kaggle/working')
!rm -r train
!mkdir train
%cd train
!mkdir images
!mkdir labels
%cd ..
!rm -r val
!mkdir val
%cd val
!mkdir images
!mkdir labels
%cd ..

In [ ]:
import shutil
from tqdm import tqdm
for i in tqdm( range(df_train.shape[0]),desc='Copying...'):
    try:
        shutil.copy(df_train.iloc[i,6],df_train.iloc[i,10])
    except:
        print(df_train.iloc[i,6],df_train.iloc[i,10])
    

In [ ]:
def update_annotations(df_old):
    annotations_lst = list()
    for i in range(df_old.shape[0]):
        temp_lst = list()
        for box in df_old.iloc[i]:
            temp_dct = dict()
            if(box['width']+box['x']>1280):
                box['width'] = 1280 - box['x']
            if(box['height']+box['y']>1280):
                box['height'] = 1280 - box['y']
            temp_dct['x_center'] = box['x'] + box['width']/2.0
            temp_dct['y_center'] = box['y'] + box['height']/2.0
            temp_dct['width'] = box['width']
            temp_dct['height'] = box['height']
            annotations_string = ' '.join(['0',str(temp_dct['x_center']/1280),str(temp_dct['y_center']/720),str(temp_dct['width']/1280),str(temp_dct['height']/720)])
            temp_lst.append(annotations_string)
        annotations_lst.append(temp_lst)
    annotations = pd.Series(annotations_lst,name='Annotations')
    return annotations
            

In [ ]:
df_train['String_Annotations'] = update_annotations(df_train['annotations'])
df_train.head(2)

In [ ]:
for i in tqdm( range(df_train.shape[0]),desc='Creating Labels...'):
    f = open(df_train.iloc[i,11],"w")
    to_string = "\n".join(df_train.iloc[i,12])
    f.write(to_string)
    f.close()

In [ ]:
def plot_image(df_train,image_id):
    image_path = df_train['new_image_path'][image_id]
    img = Image.open(image_path)
    for box in df_train['annotations'][image_id]:
        ImageDraw.Draw(img).rectangle([box['x'],box['y'],box['x']+box['width'],box['y']+box['height']],outline='red',width=4)
    display(img)

In [ ]:
plot_image(df_train,random.randint(0,4191))

In [ ]:
!rm -r yolov5
!git clone https://github.com/ultralytics/yolov5 

In [ ]:
%cd yolov5

In [ ]:
!pip install -r requirements.txt

In [ ]:
data_dct ={
"path": "/kaggle/working",
"train": "train/images",
"val": "val/images",

"nc": 1,
"names": ['COTS']
}

In [ ]:
import yaml
f = open('/kaggle/working/data.yaml','w')
document = yaml.dump(data_dct,f)
f.close()

In [ ]:
hyps = {
"lr0": 0.01,  # initial learning rate (SGD=1E-2, Adam=1E-3)
"lrf": 0.1,  # final OneCycleLR learning rate (lr0 * lrf)
"momentum": 0.937,  # SGD momentum/Adam beta1
"weight_decay": 0.0005,  # optimizer weight decay 5e-4
"warmup_epochs": 3.0,  # warmup epochs (fractions ok)
"warmup_momentum": 0.8,  # warmup initial momentum
"warmup_bias_lr": 0.1 , # warmup initial bias lr
"box": 0.05,  # box loss gain
"cls": 0.5 , # cls loss gain
"cls_pw": 1.0,  # cls BCELoss positive_weight
"obj": 1.0 , # obj loss gain (scale with pixels)
"obj_pw": 1.0,  # obj BCELoss positive_weight
"iou_t": 0.20,  # IoU training threshold
"anchor_t": 4.0,  # anchor-multiple threshold
# anchors: 3  # anchors per output layer (0 to ignore)
"fl_gamma": 0.0,  # focal loss gamma (efficientDet default gamma=1.5)
"hsv_h": 0.015 , # image HSV-Hue augmentation (fraction)
"hsv_s": 0.7 , # image HSV-Saturation augmentation (fraction)
"hsv_v": 0.4 , # image HSV-Value augmentation (fraction)
"degrees": 0.30,  # image rotation (+/- deg)
"translate": 0.10,  # image translation (+/- fraction)
"scale": 0.10  ,# image scale (+/- gain)
"shear": 2.0  ,# image shear (+/- deg)
"perspective": 0.0,  # image perspective (+/- fraction), range 0-0.001
"flipud": 0.0  ,# image flip up-down (probability)
"fliplr": 0.5  ,# image flip left-right (probability)
"mosaic": 0.2 , # image mosaic (probability)
"mixup": 0.5 ,# image mixup (probability)
"copy_paste": 0.0  # segment copy-paste (probability)
}

In [ ]:
f = open('/kaggle/working/hyp.yaml','w')
document = yaml.dump(hyps,f)
f.close()

In [ ]:
%cd ..

In [ ]:
%cd yolov5

In [ ]:
!pip install -q wandb
import wandb
wandb.login()

In [ ]:
!python train.py --img 1280\
--batch 4 \
--epochs 10 \
--data /kaggle/working/data.yaml \
--hyp /kaggle/working/hyp.yaml \
--weights yolov5m.pt \
--project 'COTSdetect' \
--name 'YOLOv5'

In [ ]:
table = df_train['Num_of_Bounding_boxes'].value_counts()
table.plot.bar()

In [ ]:
df_train['area_bbox'].plot.hist()

In [ ]:
df_train['video_id'].value_counts().plot.bar()

In [ ]:
img_data = Image.open(df_train.image_path[19670])
img_arr = np.array(img_data)
print(img_arr.shape,img_arr.max(),img_arr.min(),sep=" ")

In [ ]:
df_final = pd.DataFrame()
df_final['Image_Path'] = df_train['image_path']
df_final['boxes'] = df_train['Num_of_Bounding_boxes']
df_final.head()

In [ ]:
def annotations_to_str(annotations):
    main_lst = list()
    for i in range(annotations.shape[0]):
        temp_lst = list()
        for box in annotations.iloc[i]:
            string_annotations = str(0)+" "+str(box['x_center'])+" "+str(box['y_center'])+" "+str(box['width'])+" "+str(box['height'])
            temp_lst.append(string_annotations)
        main_lst.append(temp_lst)
    arr = pd.Series(main_lst,name="String_Annotations")
    return arr

In [ ]:
df_final['Annotations'] = update_annotations(df_train['annotations'])
df_final['String_Annotations'] = annotations_to_str(df_final['Annotations'])
df_final.head()

In [ ]:
import torch
import math

In [ ]:
def create_target(annotations,s = 19):
    arr = torch.zeros(annotations.shape[0],s,s,5)
    i = 0
    for annot in annotations:
        for box in annot:
            if(box['x_center']>=1280):
                box['x_center']=1279.9
            if(box['y_center']>=720):
                box['y_center']=719.9
            x_cell = math.floor((box['x_center']/1280.00)*s)
            y_cell = math.floor((box['y_center']/720.00)*s)
            arr[i,x_cell,y_cell,0] = 1
            arr[i,x_cell,y_cell,1] = (box['x_center']/1280.00)*s - x_cell
            arr[i,x_cell,y_cell,2] = (box['y_center']/720.00)*s - y_cell
            arr[i,x_cell,y_cell,3] = (box['width']/1280.00)*s
            arr[i,x_cell,y_cell,4] = (box['height']/720.00)*s
        i = i + 1
    return arr
            

In [ ]:
y_arr = create_target(df_final['Annotations'])
print(y_arr.shape)

In [ ]:
print(df_final['Annotations'][19670])

In [ ]:
print(y_arr[19670][9][10])

In [ ]:
def plot_image_grid(df_train,image_id):
    image_path = df_train['image_path'][image_id]
    img = Image.open(image_path)
    draw = ImageDraw.Draw(img)
    for i in range(19):
        draw.line([(1280.00/19*i, 0),(1280.00/19*i,720)] , fill=128)
        draw.line([(0, 720.00/19*i),(1280,720.00/19*i)], fill=128)
    for box in df_final['Annotations'][image_id]:
        draw.ellipse([(box['x_center']-2,box['y_center']-2),(box['x_center']+2,box['y_center']+2)],fill=500)
    for box in df_train['annotations'][image_id]:
        ImageDraw.Draw(img).rectangle([box['x'],box['y'],box['x']+box['width'],box['y']+box['height']],outline='red',width=4)
    display(img)
    

In [ ]:
plot_image_grid(df_train,19670)

In [ ]:
! git clone https://github.com/WongKinYiu/yolor

In [ ]:
!pip install --upgrade torch torchvision

In [ ]:
%cd yolor
!pip install -qr requirements.txt

In [ ]:
os.chdir("/kaggle/working")

In [ ]:
!git clone https://github.com/JunnYu/mish-cuda
%cd mish-cuda
!python setup.py build install

In [ ]:
os.chdir("/kaggle/working")

In [ ]:
!git clone https://github.com/fbcotter/pytorch_wavelets
%cd pytorch_wavelets
!pip install .

In [ ]:
os.chdir("/kaggle/working")

In [ ]:
os.makedirs("/kaggle/working/yolor_dataset/images/train")
os.makedirs("/kaggle/working/yolor_dataset/images/valid")
os.makedirs("/kaggle/working/yolor_dataset/labels/train")
os.makedirs("/kaggle/working/yolor_dataset/labels/valid")
print(f"Directory structure yor YoloR created")